In [2]:
import pyodbc
import pandas as pd
import os

conn = pyodbc.connect("DSN=clouderahive513",autocommit=True)
print(conn)

In [4]:
curr = conn.cursor()
curr.execute("use hiveassignment")
customers_df = pd.read_sql("select * from customers",conn)
orders_df = pd.read_sql("select * from orders",conn)

   customers.id customers.name customers.address  customers.salary
0             1           dave         123street              3000
1             2          ellen          45avenue              2000
2             3          kevin      62roadstreet              2222
3             4         maggie            82lane              5000
   orders.oid orders.date  orders.customer_id  orders.amount
0          23  23-03-2021                   3            210
1          12  18-04-2022                   5             67
2           4  03-12-2021                   2             89
3           5  03-12-2021                   2             22
4          45  09-09-2020                   6            333
5          63  21-10-2021                   4           1627


In [19]:
customers_df

,customers.id,customers.name,customers.address,customers.salary
0,1,dave,123street,3000
1,2,ellen,45avenue,2000
2,3,kevin,62roadstreet,2222
3,4,maggie,82lane,5000


In [18]:
orders_df

,orders.oid,orders.date,orders.customer_id,orders.amount
0,23,23-03-2021,3,210
1,12,18-04-2022,5,67
2,4,03-12-2021,2,89
3,5,03-12-2021,2,22
4,45,09-09-2020,6,333
5,63,21-10-2021,4,1627


In [9]:
curr.execute("show tables")
tables = curr.fetchall()
for t in tables:
    print(t)

('airnew', )
('airquality', )
('airquality3', )
('airquality_csv', )
('customers', )
('orders', )
('sales_order_data_csv', )
('sales_order_orc', )
('t1', )
('t2', )
('yearpart', )


In [7]:
for row in curr.columns(table='customers'):
    print("Column: ",row.column_name," , Data type: ",row.type_name,", Nullable: ",row.is_nullable)

Column:  id  , Data type:  INT , Nullable:  YES
Column:  name  , Data type:  STRING , Nullable:  YES
Column:  address  , Data type:  STRING , Nullable:  YES
Column:  salary  , Data type:  BIGINT , Nullable:  YES


In [6]:
for row in curr.columns(table='orders'):
    print("Column: ",row.column_name," , Data type: ",row.type_name,", Nullable: ",row.is_nullable)

Column:  oid  , Data type:  INT , Nullable:  YES
Column:  date  , Data type:  STRING , Nullable:  YES
Column:  customer_id  , Data type:  INT , Nullable:  YES
Column:  amount  , Data type:  BIGINT , Nullable:  YES


In [10]:
temp_table = "customer_who_shop"
curr.execute("create table if not exists "+temp_table+" as select c.id,c.name,c.address,c.salary,o.oid,o.date,o.amount from customers as c join orders as o on c.id = o.customer_id;")
curr.execute("select * from "+temp_table) 
curr.fetchall()

[(3, 'kevin', '62roadstreet', 2222, 23, '23-03-2021', 210),
 (2, 'ellen', '45avenue', 2000, 4, '03-12-2021', 89),
 (2, 'ellen', '45avenue', 2000, 5, '03-12-2021', 22),
 (4, 'maggie', '82lane', 5000, 63, '21-10-2021', 1627)]

In [12]:
joineddf = pd.read_sql("select * from "+temp_table,conn)
print(joineddf)

   customer_who_shop.id customer_who_shop.name customer_who_shop.address  \
0                     3                  kevin              62roadstreet   
1                     2                  ellen                  45avenue   
2                     2                  ellen                  45avenue   
3                     4                 maggie                    82lane   

   customer_who_shop.salary  customer_who_shop.oid customer_who_shop.date  \
0                      2222                     23             23-03-2021   
1                      2000                      4             03-12-2021   
2                      2000                      5             03-12-2021   
3                      5000                     63             21-10-2021   

   customer_who_shop.amount  
0                       210  
1                        89  
2                        22  
3                      1627  


In [13]:
curr.execute("drop table "+temp_table) 

In [14]:
curr.execute("show tables")
tables = curr.fetchall()
for t in tables:
    print(t)

('airnew', )
('airquality', )
('airquality3', )
('airquality_csv', )
('customers', )
('orders', )
('sales_order_data_csv', )
('sales_order_orc', )
('t1', )
('t2', )
('yearpart', )


In [22]:
pd.merge(customers_df,orders_df,left_on='customers.id',right_on='orders.customer_id')

,customers.id,customers.name,customers.address,customers.salary,orders.oid,orders.date,orders.customer_id,orders.amount
0,2,ellen,45avenue,2000,4,03-12-2021,2,89
1,2,ellen,45avenue,2000,5,03-12-2021,2,22
2,3,kevin,62roadstreet,2222,23,23-03-2021,3,210
3,4,maggie,82lane,5000,63,21-10-2021,4,1627


In [24]:
customers_df[customers_df['customers.salary']>2000]

,customers.id,customers.name,customers.address,customers.salary
0,1,dave,123street,3000
2,3,kevin,62roadstreet,2222
3,4,maggie,82lane,5000
